In [1]:
!pip install transformers datasets nltk rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=b86aad7d2bb714c3631eefa7dbea9336c655a45396b508272ed87c6df41c6a06
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that a

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset
from transformers import pipeline

# Load the dataset
ds = load_dataset("Shiyu-Lab/Wikipedia_Person_Unlearn", "forget_100")

# Initialize the LLaMA pipeline (ensure you have the model downloaded and set up with device=0 if using a GPU)
# pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-3B-Instruct", device=0)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.47k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/288k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/476 [00:00<?, ? examples/s]

In [7]:
import os
from datasets import load_dataset
from transformers import pipeline
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import csv
import nltk
import re

nltk.download('punkt')

MAX_NEW_TOKENS = 30
TEMPERATURE = 0.3
TOP_P = 0.9
TOP_K = 0
DO_SAMPLE = True
REPETITION_PENALTY = 1.2

rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Open a CSV file to store the outputs
with open('generated_answers_3.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [
        'Title',
        'Question',
        'Correct Answer',
        'Generated Answer',
        'BLEU Score',
        'ROUGE-L Score',
        'Exact Match',
        'F1 Score'
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Process the dataset and generate answers
    for item in ds['train']:
        # Construct the modified prompt
        prompt = (
            "Provide a short, direct answer to the following question without repeating the question or including extra information or text.\n"
            f"Question: {item['question']}\nAnswer:"
        )
        correct_answer = item['answer']

        # Generate the answer with adjusted parameters
        result = pipe(
            prompt,
            max_new_tokens=MAX_NEW_TOKENS,
            temperature=TEMPERATURE,
            top_p=TOP_P,
            top_k=TOP_K,
            repetition_penalty=REPETITION_PENALTY,
            do_sample=DO_SAMPLE
        )

        # Extract the generated text
        generated_text = result[0]['generated_text']

        # Post-process to extract the answer
        answer_start = generated_text.find('without any additional text:')
        if answer_start != -1:
            answer_text = generated_text[answer_start + len('without any additional text:'):].strip()
            answer_text = answer_text.split('\n')[0].split('.')[0].strip()
        else:
            answer_text = generated_text[len(prompt):].strip()
            answer_text = answer_text.split('\n')[0].split('.')[0].strip()

        # Normalize the correct answer and the generated answer
        def normalize_answer(s):
            """Lower text and remove punctuation, articles, and extra whitespace."""
            def remove_punctuation(text):
                return re.sub(r'\W+', ' ', text)

            return remove_punctuation(s.lower()).strip()

        reference_normalized = normalize_answer(correct_answer)
        candidate_normalized = normalize_answer(answer_text)

        # Tokenize normalized answers
        reference_tokens = nltk.word_tokenize(reference_normalized)
        candidate_tokens = nltk.word_tokenize(candidate_normalized)

        # Compute BLEU Score with smoothing
        smoothing_function = SmoothingFunction().method1
        bleu_score = sentence_bleu(
            [reference_tokens],
            candidate_tokens,
            smoothing_function=smoothing_function
        )

        # Compute ROUGE-L Score
        rouge_scores = rouge.score(reference_normalized, candidate_normalized)
        rouge_l_score = rouge_scores['rougeL'].fmeasure

        # Compute Exact Match
        exact_match = int(reference_normalized == candidate_normalized)

        # Compute F1 Score
        def f1_score(ref_tokens, cand_tokens):
            common = set(ref_tokens) & set(cand_tokens)
            num_same = len(common)

            if num_same == 0:
                return 0.0

            precision = num_same / len(cand_tokens)
            recall = num_same / len(ref_tokens)
            f1 = 2 * precision * recall / (precision + recall)
            return f1

        f1 = f1_score(reference_tokens, candidate_tokens)

        # Prepare the output dictionary
        output = {
            'Title': item['title'],
            'Question': item['question'],
            'Correct Answer': correct_answer,
            'Generated Answer': answer_text,
            'BLEU Score': f"{bleu_score:.4f}",
            'ROUGE-L Score': f"{rouge_l_score:.4f}",
            'Exact Match': exact_match,
            'F1 Score': f"{f1:.4f}"
        }

        # Write the output to the CSV file
        writer.writerow(output)

        print(
            f"Title: {output['Title']}\n"
            f"Question: {output['Question']}\n"
            f"Correct Answer: {output['Correct Answer']}\n"
            f"Generated Answer: {output['Generated Answer']}\n"
            f"BLEU Score: {output['BLEU Score']}\n"
            f"ROUGE-L Score: {output['ROUGE-L Score']}\n"
            f"Exact Match: {output['Exact Match']}\n"
            f"F1 Score: {output['F1 Score']}\n"
            "\n---\n"
        )

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Title: Benedetto Varchi
Question: What nationality was Benedetto Varchi?
Correct Answer: Italian
Generated Answer: Italian
BLEU Score: 0.1778
ROUGE-L Score: 1.0000
Exact Match: 1
F1 Score: 1.0000

---



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Title: Benedetto Varchi
Question: What professions did Benedetto Varchi have?
Correct Answer: Humanist, historian, poet
Generated Answer: Poet and historian
BLEU Score: 0.1351
ROUGE-L Score: 0.3333
Exact Match: 0
F1 Score: 0.6667

---



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Title: Louis de Rougemont
Question: What profession did Louis De Rougemont claim to be?
Correct Answer: Explorer
Generated Answer: Catholic priest
BLEU Score: 0.0000
ROUGE-L Score: 0.0000
Exact Match: 0
F1 Score: 0.0000

---



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Title: Louis de Rougemont
Question: Under what name did Louis De Rougemont write about his invented adventures?
Correct Answer: Louis De Rougemont
Generated Answer: The Master of Ballantrae
BLEU Score: 0.0000
ROUGE-L Score: 0.0000
Exact Match: 0
F1 Score: 0.0000

---



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Title: Louis de Rougemont
Question: What did Louis De Rougemont claim to have been worshipped as by Indigenous Australians?
Correct Answer: A god
Generated Answer: The Holy Spirit
BLEU Score: 0.0000
ROUGE-L Score: 0.0000
Exact Match: 0
F1 Score: 0.0000

---

Title: Louis de Rougemont
Question: Which society examined Louis De Rougemont's claims?
Correct Answer: The Royal Geographical Society
Generated Answer: The Catholic Church
BLEU Score: 0.0814
ROUGE-L Score: 0.2857
Exact Match: 0
F1 Score: 0.2857

---



In [13]:
import torch

del pipe

torch.cuda.empty_cache()

In [10]:
pipe = pipeline(
    "text-generation",
    model="nvidia/Nemotron-Mini-4B-Instruct",
    tokenizer=tokenizer,
    device=-1
)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("nvidia/Nemotron-Mini-4B-Instruct")

tokenizer_config.json:   0%|          | 0.00/180k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/18.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

In [11]:
import os
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import csv
import nltk
import re

nltk.download('punkt')

MAX_NEW_TOKENS = 30
TEMPERATURE = 0.3
TOP_P = 0.9
TOP_K = 0
DO_SAMPLE = True
REPETITION_PENALTY = 1.2

# Initialize the ROUGE scorer
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Open a CSV file to store the outputs
with open('generated_answers_4.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [
        'Title',
        'Question',
        'Correct Answer',
        'Generated Answer',
        'BLEU Score',
        'ROUGE-L Score',
        'Exact Match',
        'F1 Score'
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Process the dataset and generate answers
    for item in ds['train']:
        # Construct the modified prompt
        prompt = (
            "Provide a short, direct answer to the following question without repeating the question or including extra information or text.\n"
            f"Question: {item['question']}\nAnswer:"
        )
        correct_answer = item['answer']

        # Generate the answer with adjusted parameters
        result = pipe(
            prompt,
            max_new_tokens=MAX_NEW_TOKENS,
            temperature=TEMPERATURE,
            top_p=TOP_P,
            top_k=TOP_K,
            repetition_penalty=REPETITION_PENALTY,
            do_sample=DO_SAMPLE
        )

        # Extract the generated text
        generated_text = result[0]['generated_text']

        # Post-process to extract the answer
        answer_start = generated_text.find('without any additional text:')
        if answer_start != -1:
            # Get the text after the prompt
            answer_text = generated_text[answer_start + len('without any additional text:'):].strip()
            # Stop at the first newline or period
            answer_text = answer_text.split('\n')[0].split('.')[0].strip()
        else:
            # If the prompt continuation is not found, use the entire generated text after the prompt
            answer_text = generated_text[len(prompt):].strip()
            answer_text = answer_text.split('\n')[0].split('.')[0].strip()

        # Normalize the correct answer and the generated answer
        def normalize_answer(s):
            """Lower text and remove punctuation, articles, and extra whitespace."""
            def remove_punctuation(text):
                return re.sub(r'\W+', ' ', text)

            return remove_punctuation(s.lower()).strip()

        reference_normalized = normalize_answer(correct_answer)
        candidate_normalized = normalize_answer(answer_text)

        # Tokenize normalized answers
        reference_tokens = nltk.word_tokenize(reference_normalized)
        candidate_tokens = nltk.word_tokenize(candidate_normalized)

        # Compute BLEU Score with smoothing
        smoothing_function = SmoothingFunction().method1
        bleu_score = sentence_bleu(
            [reference_tokens],
            candidate_tokens,
            smoothing_function=smoothing_function
        )

        # Compute ROUGE-L Score
        rouge_scores = rouge.score(reference_normalized, candidate_normalized)
        rouge_l_score = rouge_scores['rougeL'].fmeasure

        # Compute Exact Match
        exact_match = int(reference_normalized == candidate_normalized)

        # Compute F1 Score
        def f1_score(ref_tokens, cand_tokens):
            common = set(ref_tokens) & set(cand_tokens)
            num_same = len(common)

            if num_same == 0:
                return 0.0

            precision = num_same / len(cand_tokens)
            recall = num_same / len(ref_tokens)
            f1 = 2 * precision * recall / (precision + recall)
            return f1

        f1 = f1_score(reference_tokens, candidate_tokens)

        output = {
            'Title': item['title'],
            'Question': item['question'],
            'Correct Answer': correct_answer,
            'Generated Answer': answer_text,
            'BLEU Score': f"{bleu_score:.4f}",
            'ROUGE-L Score': f"{rouge_l_score:.4f}",
            'Exact Match': exact_match,
            'F1 Score': f"{f1:.4f}"
        }

        writer.writerow(output)

        print(
            f"Title: {output['Title']}\n"
            f"Question: {output['Question']}\n"
            f"Correct Answer: {output['Correct Answer']}\n"
            f"Generated Answer: {output['Generated Answer']}\n"
            f"BLEU Score: {output['BLEU Score']}\n"
            f"ROUGE-L Score: {output['ROUGE-L Score']}\n"
            f"Exact Match: {output['Exact Match']}\n"
            f"F1 Score: {output['F1 Score']}\n"
            "\n---\n"
        )

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/models/nemotron/modeling_nemotron.py:131: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


Title: Benedetto Varchi
Question: What nationality was Benedetto Varchi?
Correct Answer: Italian
Generated Answer: Italian
BLEU Score: 0.1778
ROUGE-L Score: 1.0000
Exact Match: 1
F1 Score: 1.0000

---



Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


Title: Benedetto Varchi
Question: What professions did Benedetto Varchi have?
Correct Answer: Humanist, historian, poet
Generated Answer: lawyer and historian
BLEU Score: 0.1136
ROUGE-L Score: 0.3333
Exact Match: 0
F1 Score: 0.3333

---



Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


...

Title: Louis de Rougemont
Question: What profession did Louis De Rougemont claim to be?
Correct Answer: Explorer
Generated Answer: historian
BLEU Score: 0.0000
ROUGE-L Score: 0.0000
Exact Match: 0
F1 Score: 0.0000

---



Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


Title: Louis de Rougemont
Question: Under what name did Louis De Rougemont write about his invented adventures?
Correct Answer: Louis De Rougemont
Generated Answer: Nicholas Remy
BLEU Score: 0.0000
ROUGE-L Score: 0.0000
Exact Match: 0
F1 Score: 0.0000

---



Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


Title: Louis de Rougemont
Question: What did Louis De Rougemont claim to have been worshipped as by Indigenous Australians?
Correct Answer: A god
Generated Answer: god
BLEU Score: 0.0654
ROUGE-L Score: 0.6667
Exact Match: 0
F1 Score: 0.6667

---

Title: Louis de Rougemont
Question: Which society examined Louis De Rougemont's claims?
Correct Answer: The Royal Geographical Society
Generated Answer: American
BLEU Score: 0.0000
ROUGE-L Score: 0.0000
Exact Match: 0
F1 Score: 0.0000

---

